In [34]:
# Import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0-dev20241026


In [35]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-10-28 21:35:37--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.4’

train-data.tsv.4    100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2024-10-28 21:35:37 (17.8 MB/s) - ‘train-data.tsv.4’ saved [358233/358233]

--2024-10-28 21:35:37--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.4’

valid-data.tsv.4    100%[====

In [36]:
# Load and prepare the data
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]
train_labels = train_df["type"].values
test_labels = test_df["type"].values

In [37]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["msg"])

In [38]:
# Convert text to sequences and add padding
train_sequences = tokenizer.texts_to_sequences(train_df["msg"])
test_sequences = tokenizer.texts_to_sequences(test_df["msg"])
maxlen = max(train_df["msg"].apply(lambda x: len(x.split())))
train_sequences = pad_sequences(train_sequences, maxlen=maxlen, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=maxlen, padding='post')

In [39]:
# Create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 50, input_length=maxlen))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [40]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
# Train the model
model.fit(
    train_sequences,
    train_labels,
    validation_data=(test_sequences, test_labels),
    epochs=35,
    verbose="auto"
)

Epoch 1/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.8844 - loss: 0.2940 - val_accuracy: 0.9784 - val_loss: 0.0680
Epoch 2/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9882 - loss: 0.0566 - val_accuracy: 0.9885 - val_loss: 0.0456
Epoch 3/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9970 - loss: 0.0274 - val_accuracy: 0.9871 - val_loss: 0.0418
Epoch 4/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9988 - loss: 0.0244 - val_accuracy: 0.9878 - val_loss: 0.0450
Epoch 5/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9987 - loss: 0.0207 - val_accuracy: 0.9892 - val_loss: 0.0395
Epoch 6/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 1.0000 - loss: 0.0192 - val_accuracy: 0.9885 - val_loss: 0.0389
Epoch 7/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 1.0000 - loss: 0.0157 - val_accuracy: 0.9892 - val_loss: 0.0373
Epoch 8/35
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 1.0000 - loss: 0.0124 - val_accu

In [42]:
# Evaluate the model
evaluation_metrics = model.evaluate(test_sequences, test_labels, verbose="auto")
print('Evaluation Results -> Loss:', evaluation_metrics[0], ', Accuracy:', evaluation_metrics[1])

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9865 - loss: 0.0762
Evaluation Results -> Loss: 0.0660293698310852 , Accuracy: 0.9863505959510803


In [43]:
# Function to return prediction
def predict_message(pred_text):

    if not isinstance(pred_text, str) or len(pred_text.strip()) == 0:
        raise ValueError("Input must be a non-empty string.")

    seq = tokenizer.texts_to_sequences([pred_text])
    seq = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='pre')

    prediction_score = model.predict(seq)[0, 0]
    label = "ham" if prediction_score < 0.5 else "spam"

    return [prediction_score, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
[4.0182e-09, 'ham']


In [44]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
You passed the challenge. Great job!
